In [1]:
import random
import numpy as np
import torch
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate

In [2]:
dataset = load_dataset('csv', data_files='rotten_tomatoes_critic_reviews.csv')
print(dataset)

FileNotFoundError: Unable to find 'C:/Users/Katie/Downloads/AOL_DL_2.0\rotten_tomatoes_critic_reviews.csv'

In [ ]:
dataset['train'][0]

{'rotten_tomatoes_link': 'm/0814255',
 'critic_name': 'Andrew L. Urban',
 'top_critic': False,
 'publisher_name': 'Urban Cinefile',
 'review_type': 'Fresh',
 'review_score': None,
 'review_date': '2010-02-06',
 'review_content': 'A fantasy adventure that fuses Greek mythology to contemporary American places and values. Anyone around 15 (give or take a couple of years) will thrill to the visual spectacle'}

: 

In [ ]:
def map_label(example):
    if example["review_type"].lower() == "fresh":
        example["label"] = 1
    else:
        example["label"] = 0
    return example

dataset = dataset.map(map_label)
dataset

Map:   0%|          | 0/1130017 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date', 'review_content', 'label'],
        num_rows: 1130017
    })
})

: 

In [ ]:
dataset = dataset.filter(lambda x: x["review_content"] is not None and isinstance(x["review_content"], str) and len(x["review_content"].strip()) > 0)

Filter:   0%|          | 0/1130017 [00:00<?, ? examples/s]

: 

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date', 'review_content', 'label'],
        num_rows: 851368
    })
    test: Dataset({
        features: ['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date', 'review_content', 'label'],
        num_rows: 212843
    })
})

: 

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

: 

In [ ]:
def tokenize(example):
    return tokenizer(
        example["review_content"],
        truncation=True,
        padding="max_length",
        max_length=96
    )

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds

Map:   0%|          | 0/851368 [00:00<?, ? examples/s]

Map:   0%|          | 0/212843 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date', 'review_content', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 851368
    })
    test: Dataset({
        features: ['rotten_tomatoes_link', 'critic_name', 'top_critic', 'publisher_name', 'review_type', 'review_score', 'review_date', 'review_content', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 212843
    })
})

: 

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return accuracy.compute(predictions=preds, references=labels)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


: 

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert-rotten",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",
    fp16=True
)

: 

In [ ]:
len(dataset["train"])

851368

: 

In [ ]:
dataset['train'][0]

{'rotten_tomatoes_link': 'm/sleep_tight',
 'critic_name': 'Shaun Munro',
 'top_critic': False,
 'publisher_name': 'What Culture',
 'review_type': 'Fresh',
 'review_score': '4/5',
 'review_date': '2012-08-28',
 'review_content': "Luis Tosar is electrifying as every tenant's worst nightmare in this deeply unsettling, shrewdly measured slice of Hitchcock.",
 'label': 1}

: 

In [ ]:
print(dataset["train"]["label"].count(1))
print(dataset["train"]["label"].count(0))

544478
306890


: 

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\Katie\AppData\Local\Temp\ipykernel_33848\4045033473.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.240500,0.235129,0.905343
2,0.176600,0.232789,0.913316


TrainOutput(global_step=53212, training_loss=0.2233808052626748, metrics={'train_runtime': 16303.8535, 'train_samples_per_second': 104.438, 'train_steps_per_second': 3.264, 'total_flos': 8.400162486739968e+16, 'train_loss': 0.2233808052626748, 'epoch': 2.0})

: 

In [ ]:
trainer.save_model("./bert-rotten-output")
tokenizer.save_pretrained("./bert-rotten-output")

('./bert-rotten-output\\tokenizer_config.json',
 './bert-rotten-output\\special_tokens_map.json',
 './bert-rotten-output\\vocab.txt',
 './bert-rotten-output\\added_tokens.json',
 './bert-rotten-output\\tokenizer.json')

: 

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

C:\Users\Katie\AppData\Local\Temp\ipykernel_33848\3748788588.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.23278948664665222,
 'eval_model_preparation_time': 0.005,
 'eval_accuracy': 0.913316388135856,
 'eval_runtime': 457.1651,
 'eval_samples_per_second': 465.571,
 'eval_steps_per_second': 14.551}

: 

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("./bert-rotten-output")
tokenizer = AutoTokenizer.from_pretrained("./bert-rotten-output")

model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=1).item()
    
    return "POSITIVE" if pred == 1 else "NEGATIVE"

: 

In [ ]:
predict("This movie was terrible and boring.")

'NEGATIVE'

: 

In [ ]:
predict("This movie is very good")

'POSITIVE'

: 

In [ ]:
predict("this movie is exceptional, it really opens my eyes to another perspertive")

'POSITIVE'

: 